Imports

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from keras.optimizers import SGD
from keras.models import Sequential
import os
import matplotlib.pyplot as plt
from PIL import Image
import time

In [ ]:
PATH = os.path.abspath(os.getcwd())
IMG_PATH = os.path.join(PATH, 'images')
TRAIN_PATH = os.path.join(IMG_PATH, 'train')
TEST_PATH = os.path.join(IMG_PATH, 'test')
classes = ['kangaroo', 'yak']
mapping =  {animal : i for i, animal in enumerate(classes)}
inverse_mapping =  {i : animal for i, animal in enumerate(classes)}

Displaying the dataset

In [ ]:
for animal in classes:
    folder = os.path.join(TRAIN_PATH, animal)
    plt.figure(figsize=(10, 10))
    plt.title(animal.capitalize())
    for i, filename in enumerate(os.listdir(folder)[:9]):
        img_path = os.path.join(folder, filename)
        image = Image.open(img_path)
        plt.subplot(330 + 1 + i)
        plt.imshow(image)
        plt.axis('off')
    
    plt.show()

Loading the dataset to the memory

In [ ]:
def load_images(folder_path):
    photos, labels = [], []
    for subdir in os.listdir(folder_path):
        subdir_path = os.path.join(folder_path, subdir)
        if os.path.isdir(subdir_path):
            for filename in os.listdir(subdir_path):
                if filename.endswith('.jpg'):
                    img_path = os.path.join(subdir_path, filename)
                    image = load_img(img_path)
                    image_array = img_to_array(image)
                    photos.append(image_array)
                    labels.append(mapping[subdir])
    return np.asarray(photos) / 255.0, np.asarray(labels)

train_photos, train_labels = load_images(TRAIN_PATH)
test_photos, test_labels = load_images(TEST_PATH)

if not os.path.exists('train_photos.npy'):
    np.save('train_photos.npy', train_photos)
    print("Saved 'train_photos.npy'")

if not os.path.exists('train_labels.npy'):
    np.save('train_labels.npy', train_labels)
    print("Saved 'train_labels.npy'")

if not os.path.exists('test_photos.npy'):
    np.save('test_photos.npy', test_photos)
    print("Saved 'test_photos.npy'")

if not os.path.exists('test_labels.npy'):
    np.save('test_labels.npy', test_labels)
    print("Saved 'test_labels.npy'")

print(f'Training images shape: {train_photos.shape}')
print(f'Training labels shape: {train_labels.shape}')
print(f'Test images shape: {test_photos.shape}')
print(f'Test labels shape: {test_labels.shape}')

In [ ]:
train_photos = np.load('train_photos.npy')

plt.figure(figsize=(10, 10))
plt.title('Loaded Images')

for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(train_photos[i])
    plt.axis('off')

plt.show()

In [ ]:
train_photos = np.load('train_photos.npy')
train_labels = np.load('train_labels.npy')
test_photos = np.load('test_photos.npy')
test_labels = np.load('test_labels.npy')

VGG1

In [ ]:
def vgg1():
	model = Sequential()
	model.add(Input(shape=(256, 256, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='sigmoid'))
	opt = SGD(learning_rate=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

vgg1_model = vgg1()

start_time = time.time()
history = vgg1_model.fit(
    train_photos, train_labels,
    epochs=20,
    batch_size=16,
    verbose=1
)
end_time = time.time()

_, acc = vgg1_model.evaluate(test_photos, test_labels, verbose=1)
print('Test Accuracy: %.3f' % (acc * 100.0))
print('Time taken:', end_time - start_time)
vgg1_model.summary()

VGG3 without data augmentation

In [ ]:
def vgg3():
	model = Sequential()
	model.add(Input(shape=(256, 256, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='sigmoid'))
	opt = SGD(learning_rate=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

vgg3_no_aug = vgg3()

start_time = time.time()
history = vgg3_no_aug.fit(
    train_photos, train_labels,
    epochs=20,
    batch_size=16,
    verbose=1
)
end_time = time.time()

_, acc = vgg3_no_aug.evaluate(test_photos, test_labels, verbose=1)
print('Test Accuracy: %.3f' % (acc * 100.0))
print('Time taken:', end_time - start_time)
vgg3_no_aug.summary()

VGG3 with data augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow(
    train_photos, train_labels,
    batch_size=16
)

vgg3_with_aug = vgg3()

start_time = time.time()
history = vgg3_with_aug.fit(
    train_generator,
    epochs=20,
    verbose=1,
)
end_time = time.time()

_, acc = vgg3_with_aug.evaluate(test_photos, test_labels, verbose=1)
print('Test Accuracy: %.3f' % (acc * 100.0))
print('Time taken:', end_time - start_time)
vgg3_with_aug.summary()